# Build a Simple UI for your AI Application with Gradio

This notebook provides a hands-on guide to building a simple web interface for your generative AI applications using Gradio. By the end of this session, you will have a functional web app that interacts with a large language model, and you'll be able to share it with others.

**What is Gradio?**

Gradio is a Python library that makes it incredibly easy to create simple and interactive UIs for machine learning models and data science projects. It's particularly well-suited for use in environments like Google Colab, where you can run and share your apps with minimal setup.

**Learning Objectives:**

* Understand the basics of Gradio and its core components.
* Learn how to create a simple UI for a function that calls the Gemini API.
* See how to launch and share a Gradio app directly from a Colab notebook.

## 1. Setup and Authentication

First, we need to authenticate to use the Google Cloud services and install the necessary Python libraries. The following cell will:

1.  Authenticate you in the Colab environment.
2.  Install the `google-generativeai` library to interact with the Gemini API.
3.  Install the `gradio` library for building the user interface.

In [1]:
import os
import sys
from pathlib import Path

IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    !pip install -q google-genai google-auth google-api-core python-dotenv numpy chromadb gradio
    from google.colab import auth
    auth.authenticate_user()
    try:
        PROJECT_ID = input("Enter your Google Cloud Project ID (press Enter to use default ADC): ").strip()
    except Exception:
        PROJECT_ID = ""
    if PROJECT_ID:
        os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
else:
    def find_service_account_json(max_up=6):
        p = Path.cwd()
        for _ in range(max_up):
            candidate = p / "series-2-coding-llms" / "creds"
            if candidate.exists():
                for f in candidate.glob("*.json"):
                    return str(f.resolve())
            candidate2 = p / "creds"
            if candidate2.exists():
                for f in candidate2.glob("*.json"):
                    return str(f.resolve())
            p = p.parent
        return None

    sa_path = find_service_account_json()
    if sa_path:
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = sa_path
    else:
        try:
            from dotenv import load_dotenv
            load_dotenv()
        except Exception:
            pass


## 2. Initialize the Generative AI Client

Now that we're authenticated and have the libraries installed, we need to set up our project and initialize the client for the Generative AI service. This will allow our application to communicate with the Gemini model.

In [2]:
import os
from google import genai
from google.genai import types
import google.auth
from google.api_core import exceptions
import gradio as gr

creds, project = google.auth.default()
project = os.environ.get("GOOGLE_CLOUD_PROJECT", project)
client = genai.Client(vertexai=True, project=project, location="us-central1")
print(f"Using project: {project}")

print("✅ Environment loaded successfully!")

Using project: coding-with-ai-wn-26
✅ Environment loaded successfully!


## 3. Create a Function to Interact with the Model

This is the core logic of our application. We'll create a Python function that takes a user's prompt as input, sends it to the Gemini model, and returns the model's response. This function will be the backend for our Gradio UI.

Of course, this is a primitive little function - you could extend it with any of the tools you've built

In [3]:
def get_gemini_response(prompt_text):
  """Sends a prompt to the Gemini model and returns the response."""
  try:
    response = client.models.generate_content(
    model="gemini-2.5-flash-lite",
    contents=prompt_text
    )

    return response.text
  except Exception as e:
    return f"An error occurred: {e}"

## 4. Build and Launch the Gradio Interface

Now for the exciting part! We'll use Gradio to create a web interface for our `get_gemini_response` function. With just one line of code, we can create an interface with a textbox for input and a textbox for the output.

When you run the cell below, Gradio will create and launch the interface. You'll see it directly in the notebook output, and you'll also get a public URL that you can share with anyone for 72 hours.

In [4]:
iface = gr.Interface(
    fn=get_gemini_response, 
    inputs=gr.Textbox(lines=5, label="Your Prompt"), 
    outputs=gr.Textbox(label="Gemini's Response"),
    title="Simple Interface for Gemini",
    description="Ask Gemini anything! Enter your prompt below and see the response."
)

iface.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
